In [2]:
import numpy as np
import pandas as pd
import os
import pickle
import math
import pdb
from matplotlib import pyplot as plt
import importlib

from cartopy import crs as ccrs
from cartopy import feature as cfeature
from matplotlib import cm

import loaders
import geography
import visualization

importlib.reload(loaders)
importlib.reload(geography)
importlib.reload(visualization)
import visualization as vis

%matplotlib

Using matplotlib backend: Qt5Agg


In [4]:
df = pd.read_csv('tables/global_risks_rep_wuhan3_kappa1.csv').sort_values('risk_i', ascending=False)
df = loaders.desc_from_iata_code(df, 'Origin')#.set_index('Origin', drop=True)
india = df[df.region.str.contains('IN').fillna(False)].reset_index(drop=True)
africa =  df[df.continent == 'AF'].reset_index(drop=True)

print(len(df), len(africa), len(india))

1364 153 77


In [27]:
import math
import numpy as np
def get_p2(p1, n1, n2, q=1e-6):
    enum = np.power( (1 - q*(1-p1)), n2/n1) - 1
    return 1 + enum / q

def f(p, n, q):
    return (1 - q * (1-p))**n

p1 = 0.01
n1 = 10000
n2 = 20009
q = 1e-6
p2 = get_p2(p1=p1, n1=n1, n2=n2, q=q)

for q in [1e-4, 1e-5, 1e-6, 1e-7, 1e-8]:
    print(f(p1, n1, q) - f(p2, n2, q))

0.3525706212972674
0.23297794801271188
0.029009289218990708
0.0029662085213627565
0.00029728264841577534
